<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-335 Tecnologías de Búsqueda en la Web</h1>

<H3 align='center'> Análisis de Comentarios en Twitter Respecto al Acontecer Nacional Actual </H3>
<hr style="height:2px;border:none"/>


In [32]:
import twint 
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize, WordNetLemmatizer

## Lectura de Tweets

In [22]:
df = pd.read_csv('test_data.csv', encoding='utf-8')
df.drop(columns=['hashtags', 'geo', 'source'], inplace=True)
df

,date,timezone,user_id,username,tweet
0,2019-10-18,-3,70858061,danrod_cl,Mira lo que está pasando en éste momento en Sa...
1,2019-10-18,-3,1084048632859557889,paulale93970029,#EvasionMasiva #EvasionMasivaTodoElDia FUERZA...
2,2019-10-18,-3,1116124606547537930,sjwkekskkw,wey ya al chile explicame xq no entiendo
3,2019-10-18,-3,4323083121,karameelahs,"whew, a mess chile."
4,2019-10-18,-3,1183471425295962113,vbetmar,Cara de sinica a esta vieja chile tracionera v...
...,...,...,...,...,...
9521,2019-10-18,-3,39692424,alertanews24,🇨🇱 | URGENTE - PROTESTAS EN CHILE: Gran cantid...
9522,2019-10-18,-3,996238739184799745,educavialchile,✅SISTEMA DE RETENCIÓN INFANTIL✅\nExpresa el am...
9523,2019-10-18,-3,1006624953519820801,vampperro,Somos de Santiago de Chile
9524,2019-10-18,-3,46149982,superarpobreza,🎈🏡¡Este domingo 20 de octubre celebra con tus ...


## Limpieza de Datos

In [28]:
# Función para quitar emojis
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [29]:
df["tweet"] = df["tweet"].str.lower()

tweets=df['tweet'].apply(deEmojify)
tweets

0       mira lo que est pasando en ste momento en sant...
1       #evasionmasiva #evasionmasivatodoeldia  fuerza...
2                wey ya al chile explicame xq no entiendo
3                                     whew, a mess chile.
4       cara de sinica a esta vieja chile tracionera v...
                              ...                        
9521     | urgente - protestas en chile: gran cantidad...
9522    sistema de retencin infantil\nexpresa el amor ...
9523                           somos de santiago de chile
9524    este domingo 20 de octubre celebra con tus vec...
9525    el listado todos contra todos de los 10 vehcul...
Name: tweet, Length: 9526, dtype: object

In [30]:
def lemmatize(word, corpus='nltk'):
    if corpus == 'nltk':
        lemmatizer = WordNetLemmatizer()
        return lemmatizer.lemmatize(word)

def tokenize(data, corpus='nltk'):
    stop_words = stopwords.words('spanish')
    lemmatizer = WordNetLemmatizer()
    tokenize_data = []
    for text in data:
        word_tokens = word_tokenize(text)
        words = ''
        for word in word_tokens:
            if (word not in stop_words) and len(word) > 3:
                if corpus == 'nltk':
                    words += ' ' + lemmatize(word)
        tokenize_data += [words]
    return tokenize_data

In [31]:
tokenize(tweets)

[' mira pasando momento santiago chile caos',
 ' evasionmasiva evasionmasivatodoeldia fuerza chile pic.twitter.com/sw1duwyimm',
 ' chile explicame entiendo',
 ' whew mess chile',
 ' cara sinica vieja chile tracionera vamos huella muestra prensa pura',
 ' viven burbuja creen chile desastre verdad mejor vivir latinoamrica',
 ' ustedes preocupados chile http //twitter.com/biobio/status/1185335677724090373',
 ' continuan evasiones metro chile http //youtu.be/za3ty6aazua youtube',
 ' whatttttt ummm chile anywayssss',
 ' ganas chile pueblo can abuso',
 ' muni_provi deprovidencia sosafe_cl pdi_chile http //twitter.com/inagroovie/status/1185341495408320512',
 ' 18oct santiago chile ciudadanos vieron obligados caminar regresar hogares tras cierre toda metrosantiago fuertes disturbios video cortesa pic.twitter.com/yj14ou8bwn',
 ' ustedes peores hace dcadas ustedes polticos derecha izquierda estn haciendo chile pobre busque charco disfrute vida necesidades mantenida habitantes chile pic.twitter.c